In [ ]:
import pandas as pd
from google.colab import drive
import numpy as np
from datetime import datetime

drive.mount('/content/drive')

path = '/content/drive/MyDrive/allBabyNamesUSA_2000s.csv'

import pandas as pd
df = pd.read_csv(path)
df

Mounted at /content/drive


,Rank,Name,Number,Gender
0,1,Emily,223249,female
1,2,Madison,192756,female
2,3,Emma,180786,female
3,4,Olivia,155608,female
4,5,Hannah,155331,female
...,...,...,...,...
19995,9996,Davyd,55,male
19996,9997,Deklyn,55,male
19997,9998,Donivin,55,male
19998,9999,Dontray,55,male


In [ ]:
import pandas as pd
import re

# Regex patterns for names
full_name_pattern = re.compile(r'^[A-Z][a-z]+\s[A-Z][a-z]+$', re.IGNORECASE)  # "John Doe"
single_name_pattern = re.compile(r'^[A-Z][a-z]+$', re.IGNORECASE)  # "John" or "Doe"

# Common non-name values (race, gender labels, etc.)
non_name_values = {"male", "female", "nonbinary", "black", "white", "asian", "hispanic", "latino", "native"}

# Function to refine name column detection
def find_name_columns(df):
    name_columns = []

    for column in df.columns:
        values = df[column].astype(str)

        match_count_full = values.apply(lambda x: bool(full_name_pattern.match(x))).sum()
        match_count_single = values.apply(lambda x: bool(single_name_pattern.match(x))).sum()
        non_name_count = values.apply(lambda x: x.lower() in non_name_values).sum()

        # Check if values are consistently short (likely a name, not descriptive text)
        avg_word_count = values.apply(lambda x: len(str(x).split())).mean()

        # Consider a column a "name column" if:
        # - At least 70% of values match name patterns
        # - Less than 10% match race/gender labels
        # - Word count average is **≤ 2** (filters out descriptive categorical data)
        if max(match_count_full, match_count_single) / len(values) > 0.7 and non_name_count / len(values) < 0.1 and avg_word_count <= 2:
            name_columns.append(column)

    return name_columns

# Identify columns containing names
name_columns = find_name_columns(df)

# Print results
if name_columns:
    print(f"✅ Name columns found: {name_columns}")
else:
    print("❌ No name columns detected.")

✅ Name columns found: ['Name']


In [ ]:
import pandas as pd


# Function to extract only the first letter of names
def first_letter_only(name):
    return name[0] if isinstance(name, str) and name.strip() != "" else ""  # Ensures valid input

# Apply transformation **only to first 50 rows**
df.loc[:49, name_columns] = df.loc[:49, name_columns].applymap(first_letter_only)

# Print results
print(df.head(50))  # Show first 50 modified rows

    Rank Name  Number  Gender
0      1    E  223249  female
1      2    M  192756  female
2      3    E  180786  female
3      4    O  155608  female
4      5    H  155331  female
5      6    A  150518  female
6      7    I  148996  female
7      8    S  133979  female
8      9    E  132905  female
9     10    A  132805  female
10    11    A  130448  female
11    12    S  123932  female
12    13    S  118838  female
13    14    A  114231  female
14    15    G  110389  female
15    16    A  104241  female
16    17    T  100797  female
17    18    B   99523  female
18    19    L   97011  female
19    20    C   96189  female
20    21    N   94853  female
21    22    K   94108  female
22    23    J   90460  female
23    24    A   89867  female
24    25    V   84701  female
25    26    M   83276  female
26    27    H   81767  female
27    28    S   76328  female
28    29    J   75797  female
29    30    J   70725  female
30    31    M   70634  female
31    32    D   69496  female
32    33  

<ipython-input-3-1033100105>:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df.loc[:49, name_columns] = df.loc[:49, name_columns].applymap(first_letter_only)


In [ ]:
import pandas as pd

# Function to classify individual values
def classify_name(value):
    value = str(value).strip()
    if len(value) == 1:  # Single character = Initial
        return "Initial"
    elif len(value) > 1:  # More than one character = Full Name
        return "Full Name"
    else:
        return "Unknown"


# Apply classification function to each value
for col in name_columns:
    df[f"{col} Type"] = df[col].apply(classify_name)

# Count full names and initials
for col in name_columns:
    full_names_count = (df[f"{col} Type"] == "Full Name").sum()
    initials_count = (df[f"{col} Type"] == "Initial").sum()

    # Print results
    print(f"❌ {col}: {full_names_count} full names, {initials_count} initials, Data is inconsistent. Suggest getting the full name for values with just initials.")

❌ Name: 19950 full names, 50 initials, Data is inconsistent. Suggest getting the full name for values with just initials.
